In [1]:
import pandas as pd
import numpy as np

import sklearn.datasets

##Seaborn for fancy plots. 
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams["figure.figsize"] = (8,8)

# Pipelines

Sklearn provides a tool for stiching together some of the data preprocessing steps that go into preparing our data, such as encoding categorical variables and rescaling numerical varaibles - the pipeline. 

The pipeline is concpetually pretty simple, we are just doing the same steps that we'd normally do to perform these data-prep functions. Probably the most challenging change is that the pipeline is more abstract - we don't see each step explicitly laid out, we set them all up and then just get results back. For this reason, it is important that we're comfortable with the ideas as there are fewer steps where we can troubleshoot. 

The basic process for using a pipeline is:
<ul>
<li>Create the pipeline object, which is imported from sklearn. 
<li>Set each preparation step, and the model step as arguments when making the pipeline. 
<li>Fit the pipeline object (instead of the model) with the data. (The pipeline roughly takes the place of the model)
<li>Score and predict with the pipeline object. 
<li>Documentation is here: https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html 
</ul>

For example, we will take our dataset and rescale variables, before using a tree classifier

In [2]:
#Simple Pipeline
df1 = pd.read_csv("data/diabetes.csv")
df1.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


We have a categorical target and some numerical features. 

The pipeline will be to rescale the features, then train the model. 

In [3]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

#Split data as normal
y = np.array(df1["Outcome"]).reshape(-1,1)
X = np.array(df1.drop(columns={"Outcome"}))
X_train, X_test, y_train, y_test = train_test_split(X, y)

#Build pipeline
pipeline_steps = [('scaler', StandardScaler()),('DT', DecisionTreeClassifier()) ]
pipe = Pipeline(pipeline_steps)
# The pipeline can be used as any other estimator

pipe.fit(X_train, y_train)

pipe.score(X_test, y_test)

0.71875

Voila! Clean and simple!

We could do the exact same thing with encoding if we had categorical varaibles. 

In [4]:
df2 = pd.read_csv("data/titanic_train.csv")
df2.drop(columns={"PassengerId", "Name", "Ticket", "Fare", "Cabin", "Age", "SibSp", "Parch"}, inplace=True)
df2.head()

,Survived,Pclass,Sex,Embarked
0,0,3,male,S
1,1,1,female,C
2,1,3,female,S
3,1,1,female,S
4,0,3,male,S


In [5]:
for i in df2.columns:
    df2[i] = df2[i].astype('category')
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   Survived  891 non-null    category
 1   Pclass    891 non-null    category
 2   Sex       891 non-null    category
 3   Embarked  889 non-null    category
dtypes: category(4)
memory usage: 4.1 KB


In [6]:
from sklearn.preprocessing import OneHotEncoder

#Split data as normal
y = np.array(df2["Survived"]).reshape(-1,1)
X = np.array(df2.drop(columns={"Survived"}))
X_train, X_test, y_train, y_test = train_test_split(X, y)

#Build pipeline
pipeline_steps = [('ohe', OneHotEncoder()),('DT', DecisionTreeClassifier()) ]
pipe = Pipeline(pipeline_steps)
# The pipeline can be used as any other estimator
# and avoids leaking the test set into the train set
pipe.fit(X_train, y_train)

pipe.score(X_test, y_test)

0.8071748878923767

## Combined Variable Types

One thing we didn't handle "properly" above was the numerical values, we just dropped them. In order to deal with feature sets that have both categorical (requires encoding) and numerical (requires scaling) we need to use something slightly more complex - the Column Transformer. 

Our data processing pipeline will usually need to involve several different steps on several different features, some may need to be encoded, others may need to be scaled, and others may need to have missing values dealt with. The Column Transformer is a tool that allows us to split our data processing pipeline into several different paths, and send the appropriate subset of data into each of those paths. 

We'll look at how we can use a column transformer after a quick detour into one thing we commonly want to use inside of one, imputation...

### Imputation

One common thing that we need to deal with in processing data for modelling is to handle data that is missing, and extending from the idea of the column transformer it is common that for different features we may need to deal with those missing values differently. Until now we have dealt with any missing data by just deleting the row, so there are no missing values; this action is a simplified version of a process called imputation. Imputation is the general process of filling in missing values. In this data we can see from the Describe that there are 891 rows, but under Age there are some missing values. We could delete these rows, or we could <b><i>impute</i><b> a value to insert as a placeholder - or generate a value to plug in, replacing the missing value. This allows us to keep those rows that have a missing value. 

Imputation is a very common data preparation step, it takes blank data and "deals with it". Dealing with it may mean removing those blanks, or using some more advanced analysis to determine a placeholder value to insert. 

In [7]:
df3 = pd.read_csv("data/titanic_train.csv")
df3.drop(columns={"PassengerId", "Name", "Ticket", "Cabin"}, inplace=True)
df3.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [8]:
df3.describe().T

,count,mean,std,min,25%,50%,75%,max
Survived,891.0,0.383838,0.486592,0.00,0.0000,0.0000,1.0,1.0000
Pclass,891.0,2.308642,0.836071,1.00,2.0000,3.0000,3.0,3.0000
Age,714.0,29.699118,14.526497,0.42,20.1250,28.0000,38.0,80.0000
SibSp,891.0,0.523008,1.102743,0.00,0.0000,0.0000,1.0,8.0000
Parch,891.0,0.381594,0.806057,0.00,0.0000,0.0000,0.0,6.0000
Fare,891.0,32.204208,49.693429,0.00,7.9104,14.4542,31.0,512.3292


### Imputation Considerations

The positive to doing this imputation is that we don't loose data. In many real world scenarios we often have some data that is missing. Think about a scenario where we are attempting to predict if people will get skin cancer - our features may be things like where they live, sun exposure, outdoor jobs, tanning, race/skin color, medical info, etc... In a real world study it is pretty difficult to gather a large dataset where every useful feature can be captured for every row in the dataset. Deleting every row where we have any missing value would likely cut our large, and hard and expensive to gather data, down to a shell of itself - if we did a large cancer study that spanned years, we probably wouldn't want to delete patients if they failed to complete one survey question. By imputing those missing values we can keep our valuable data by plugging in some placeholder for those occasional missing spots. 

For this example, we are plugging in an average (mean) age for the rows of Titanic-ers for which we have no age. This seems somewhat reasonable based on intuition - if there is some random passenger and we have no idea of their age, but we assume they are ~30, that probably doesn't mess things up too dramatically. There's a possibility that they are really old and we are way off, but the 75% percentile is 38, so the odds of that are pretty low. Overall we are probably 'winning' by keeping all that other data by using the imputation of age. 

The negative is that we are literally inventing a new value, and plugging it in - we are making up fake data. This can obviously introduce error if we do it poorly. For example, assume we are examining stock market data to predict stock price increases, to determine if we should invest. Imputing profitability (earnings per share) may not be super logical in this scenario. 

#### Why is it Missing?

One key consideration in the real world in doing imputation is examining why the data is missing. Is the data just randomly missing, or is there some reason why some values may be missing?

For example, survey data that involves people self-identifying with/as something that may incur discrimination (lgbt, race, disability, age) are commonly underidentified. Collecting data may be more challenging in remote environments vs cities. Forms may be poorly laid out, leading to one question being consistently overlooked. 

There are a lot of reasons that values may be missing, and examination with domain knowledge will help us determine the most appropriate imputation strategy. We will look at more sophisticated imputation stuff later on. 

#### Imputation Overall and Initial Strategy

Imputation is extremely common in machine learning. In the smaller test datasets that we often use it isn't as important, as these tend to be more complete. In real world scenarios, especially when data is hard to collect (e.g. census, suveys, etc...), it is pretty common and often required to maintain the usability of our datasets. 

#### Imputation Approach

For now, we can focus on simple imputation:

<ul>
<li>Numerical Imputation:
<ul>
<li>Replace with median: If we want to maintain median - outliers, skewed distributions such as income. 
<li>Replace with mean: If we want to maintain mean - normal(ish) distributions such as height. 
</ul>
<li>Categorical Imputation:
<ul>
<li>Replace with mode (most common value)
</ul>
<li>Either:
<ul>
<li>If there is some default value, we can fill with a constant. For example, if you have data on number of citizenships that people have, the vast majority have 1. It makes sense to impute 1 if that data is missing (in this case that's the mode). 
</ul>
</ul>

There are several more complex imputation techniques that can be used, we'll look at some briefly when we look at clustering. The basic idea behind all of them is that they seek to impute a value that is "better" than one of the simple guesses by using the other data to come up with a more targeted value to impute. For example, if we were imputing a value for someone's income, and we knew their profession, it would likely make sense to impute a different value for the income of a brain surgeon than for a parking lot attendant. 

One more consideration is the number of missing values in a column - if we are just imputing a huge percentage of the values for one of our features, that feature probably isn't helping us all that much. A rule of thumb is that if you're missing about 20%-30% of the values in a column it is better to just drop that column rather than impute values for it. This rule of thumb is pretty loose - it really requires a specific look at the data, mainly the reason that values are missing. If we can infer some meaning from what's missing, we are more likely to be able to determine a logical imputation. In this Titanic example, Age is missing in a large number of rows, but keeping it is still worth it because the imputation is reasonable. 

Lastly, fillna in pandas also imputes similarly to what we are doing here. 

## Pipelines with Column Transformers

Our goal here is to create a pipleine to do the following:
<ul>
<li>Impute missing numeric values with the mean. 
<li>Scale numeric values with a standard scaler. 
<li>Encode categorical values.
<li>Model with a decision tree. 
</ul>

The column transformer object allows us to split the features that we are processing and deal with each group separately, this is very common in situations where we have both numeric and categorical variables to prepare - the numeric varaibles can be scaled, the categorical ones can be encoded, then they can be recombined into our prepared dataset.

![Column Transformer](images/col_trans.png "Column Transformer")

### Using a Column Transformer

A column transformer functions like a slightly more elaborate pipeline:
<ul>
<li> The column transformer takes in multiple data subsets, and preprocessing pipelines. 
    <ul>
    <li> Each argument in creating the CT contains 3 things - an informal label, the columns to process, and a pipeline of the processing steps. 
    </ul>
<li> Each subset flows through the supplied processing pipeline. 
<li> At the end, the data is recombined, and the data is spit out ready for modelling. 
<li> In general, when using a column transformer, we will have:
    <ul> 
    <li> A column transformer that is effectively "the preprocessing pipeline". 
    <li> Within that transformer, a pipeline that deals with each distinct subset of data (usually one for categorical, one for numerical).
    <li> Another pipeline with two steps - the column transformer and the model, this is what we call "fit" on. 
    </ul>
</ul>
<b>Note:</b> we can split the columns into our groups in the column transformer either by manual inspection, or by some automatic filter like the data type. If you're automating it, make sure that you can trust that the data types are trustworthy - something like a rouge "n/a" in a numerical column could mess everything up. 

This all sounds a bit complex, but it isn't much worse than setting up a pipeline, here we've just layered in another step and we need to split the data into subsets. The steps like fit and score that we are used to with pipelines or models all work the same once the column transformer is established. 

In [9]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

numeric_features = ["Age", "Fare", "SibSp", "Parch"]
numeric_transformer = Pipeline( steps=[
        ("imputer", SimpleImputer(strategy="mean")),
        ("scaler", StandardScaler())
    ])

categorical_features = ["Embarked", "Sex", "Pclass"]
categorical_transformer = OneHotEncoder()

preprocessor = ColumnTransformer( transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ])

Now that the column transformer is created, putting it to use is a familiar process. 

In [10]:
#Split data as normal
y = df3["Survived"]
X = df3.drop(columns={"Survived"})

#Build pipeline
pipeline_steps = [('pre', preprocessor),('DT', DecisionTreeClassifier()) ]
pipe = Pipeline(pipeline_steps)

X_train, X_test, y_train, y_test = train_test_split(X, y)
# The pipeline can be used as any other estimator
# and avoids leaking the test set into the train set

pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

0.7668161434977578

#### Combining Pipeline Objects

As you may have picked up on, there's a lot of interchangeability between the sklearn objects that we use to prepare and model data. This is generally good, as we can arrange the exact pieces that we need and they'll work together in almost any combination. This can also be a little confusing, as there's not one fixed way to do things. In the long run, we'll get more used to this and be able to use it to our advantage - as long as something fits the template of these preprocessing tools (there are specific classes to extend, we'll look at it a little more later on) we can plug and play different preprocessing items, and even create our own. 

The ability to swap things in and out will be useful soon when we look at a grid search - the interchangability of steps helps allow us to automate the running of different trials to see what is most accurate. 

## Exercise - Predict the Target

In [11]:
df_ = pd.read_csv("data/diabetes.csv")
df_.rename(columns={"Outcome":"target"}, inplace=True)
df_.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,target
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [12]:
df_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   target                    768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


### Create Column Transformer and Model

In [13]:
numeric_features_ = ["Pregnancies", "Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI", "DiabetesPedigreeFunction", "Age"]
numeric_transformer_ = Pipeline( steps=[
        ("imputer", SimpleImputer(strategy="mean")),
        ("scaler", StandardScaler())
    ])

categorical_features_ = []
categorical_transformer_ = OneHotEncoder()

preprocessor_ = ColumnTransformer( transformers=[
        ("num", numeric_transformer_, numeric_features_),
        ("cat", categorical_transformer_, categorical_features_),
    ])

#Split data as normal
#y = np.array(df3["Survived"]).reshape(-1,1)
#X = np.array(df3.drop(columns={"Survived"}))
y_ = df_["target"]
X_ = df_.drop(columns={"target"})

#Build pipeline
pipeline_steps_ = [('pre', preprocessor_),('DT', DecisionTreeClassifier()) ]
pipe_ = Pipeline(pipeline_steps_)

X_train_, X_test_, y_train_, y_test_ = train_test_split(X_, y_)
# The pipeline can be used as any other estimator
# and avoids leaking the test set into the train set

pipe_.fit(X_train_, y_train_)
print(pipe_.score(X_test_, y_test_))


0.6614583333333334
